## 模型调参  Baseline:0.703763  0.704483

1、加入原始所有特征后,  0.707527

2、进行卡方检验降维后， 0.711182， 0.712489 0.712085  0.709924

2、直接采用hyperot进行调参

In [207]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

data = pd.read_csv('./datasets/train_data2.csv')
target = pd.read_csv('./datasets/train_target2.csv', names=['label'], header=None)

data.drop(data.columns[[0]], axis=1, inplace=True)  #删除某列函数
target = target.reset_index(drop=True)

one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility', 'education','gender','house','os','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']



for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [208]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
# split = StratifiedShuffleSplit(n_splits=1, test_size=0.12, random_state=42)
y = np.array(target).squeeze()
# for train_index, test_index in split.split(data,  y):
#     X_train = data.iloc[train_index]
#     X_test = data.iloc[test_index]
#     y_train = y[train_index]
#     y_test = y[test_index]
X_train,X_test, y_train, y_test = train_test_split(data, y, test_size=0.3,random_state=2018)

In [209]:
data_clicked = X_train[y_train == 1]

In [210]:
# 处理联网类型特征
ct_train = X_train['ct'].values
ct_train = [m.split(' ') for m in ct_train]
ct_trains = []
for i in ct_train:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_trains.append(index)

ct_test = X_test['ct'].values
ct_test = [m.split(' ') for m in ct_test]
ct_tests = []
for i in ct_test:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_tests.append(index)

In [211]:
num_ad = X_train['aid'].value_counts().sort_index()
num_ad_clicked = data_clicked['aid'].value_counts().sort_index()

ratio = num_ad_clicked / num_ad

ratio_clicked = pd.DataFrame({
    'aid': ratio.index,
    'ratio_clicked' : ratio.values
})
X_train = pd.merge(X_train, ratio_clicked, on=['aid'], how='left')
X_test = pd.merge(X_test, ratio_clicked, on=['aid'], how='left')

In [212]:
num_advertise_touser = X_train.groupby('aid').uid.nunique()
num_advertise_touser = pd.DataFrame({
    'aid': num_advertise_touser.index,
    'num_advertise_touser' : num_advertise_touser.values
})
X_train = pd.merge(X_train, num_advertise_touser, on=['aid'], how='left')
X_test = pd.merge(X_test, num_advertise_touser, on=['aid'], how='left')

In [213]:
num_campaign = X_train['campaignId'].value_counts().sort_index()
num_campaign_clicked = data_clicked['campaignId'].value_counts().sort_index()

ratio_num_campaign = num_campaign_clicked / num_campaign

ratio_num_campaign = pd.DataFrame({
    'campaignId': ratio_num_campaign.index,
    'ratio_num_campaign' : ratio_num_campaign.values
})
X_train = pd.merge(X_train, ratio_num_campaign, on=['campaignId'], how='left')
X_test = pd.merge(X_test, ratio_num_campaign, on=['campaignId'], how='left')

分析每个广告对应的人群年龄分布, 教育水平分布，消费能力分布

In [214]:
def get_ad_toother(typename):
    num_ad_totype = X_train.groupby('aid')[typename].value_counts()
    num_ad_totype_clicked = data_clicked.groupby('aid')[typename].value_counts()
    ratio_num_ad_totype = num_ad_totype_clicked / num_ad_totype
    list_num_ad_totype = []
    num_adid = X_train['aid'].value_counts().sort_index().index
    for aid_out in num_adid:
        dict_buf = {}
        dict_num_ad_totype = {}
        dict_num_ad_totype['aid'] = aid_out
        for i, j in ratio_num_ad_totype.items():
            aid = i[0]
            feature = i[1]
            if(aid == aid_out):
                dict_buf[feature] = float("%.5f" % j)
        fea_name = 'num_ad_to'+typename
        dict_num_ad_totype[fea_name] = dict_buf
        list_num_ad_totype.append(dict_num_ad_totype)
    return list_num_ad_totype

In [96]:
list_num_ad_toedu = get_ad_toother('education')
list_num_ad_toedu = pd.DataFrame(list_num_ad_toedu)
X_train = pd.merge(X_train, list_num_ad_toedu, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_toedu, on=['aid'], how='left')
X_train['ratio_num_ad_toedu'] = [ j.get(i, 0) for i, j in X_train[['education', 'num_ad_toeducation']].values]
X_test['ratio_num_ad_toedu'] = [ j.get(i, 0) for i, j in X_test[['education', 'num_ad_toeducation']].values]

In [215]:
list_num_ad_toage = get_ad_toother('age')
list_num_ad_toage = pd.DataFrame(list_num_ad_toage)
X_train = pd.merge(X_train, list_num_ad_toage, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_toage, on=['aid'], how='left')
X_train['ratio_num_ad_toage'] = [ j.get(i, 0) for i, j in X_train[['age', 'num_ad_toage']].values]
X_test['ratio_num_ad_toage'] = [ j.get(i, 0) for i, j in X_test[['age', 'num_ad_toage']].values]

In [216]:
list_num_ad_toconsume = get_ad_toother('consumptionAbility')
list_num_ad_toconsume = pd.DataFrame(list_num_ad_toconsume)
X_train = pd.merge(X_train, list_num_ad_toconsume, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_toconsume, on=['aid'], how='left')
X_train['ratio_num_ad_toconsume'] = [ j.get(i, 0) for i, j in X_train[['consumptionAbility', 'num_ad_toconsumptionAbility']].values]
X_test['ratio_num_ad_toconsume'] = [ j.get(i, 0) for i, j in X_test[['consumptionAbility', 'num_ad_toconsumptionAbility']].values]

In [217]:
list_num_ad_tohouse = get_ad_toother('house')
list_num_ad_tohouse = pd.DataFrame(list_num_ad_tohouse)
X_train = pd.merge(X_train, list_num_ad_tohouse, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_tohouse, on=['aid'], how='left')
X_train['ratio_num_ad_tohouse'] = [ j.get(i, 0) for i, j in X_train[['house', 'num_ad_tohouse']].values]
X_test['ratio_num_ad_tohouse'] = [ j.get(i, 0) for i, j in X_test[['house', 'num_ad_tohouse']].values]

In [218]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [219]:
def get_common_interest(type_name, ratio):
    num_adid = data_clicked['aid'].value_counts().sort_index().index
    num_aid_clicked = dict(data_clicked['aid'].value_counts().sort_index())
    num_user_clicksameAd_interest = data_clicked.groupby('aid')[type_name].value_counts()
    dict_interest = {}
    for adid in num_adid:
        dict_buf = {}
        for interest in num_user_clicksameAd_interest.items():
            index = interest[0]
            if index[0] == adid:
                number = interest[1]
                detail = index[1]
                detail = detail.split(' ')
                for det in detail:
                    if det not in dict_buf:
                        dict_buf[det] = number
                    else:
                        dict_buf[det] += number
        dict_interest[adid] = dict_buf
    dict_common_interest = []
    for adid, dict_inter in dict_interest.items():
        dict_common_buf = {}
        dict_common_buf['aid'] = adid
        common_inter = []
        ad_total = num_aid_clicked[adid] - dict_inter.get('-1', 0)
        if '-1' in dict_inter:
            dict_inter.pop('-1')
        for id_inter, num in dict_inter.items():
            if num >= ad_total*ratio:
                common_inter.append(id_inter)
        str_name = 'common_'+type_name
        dict_common_buf[str_name] = common_inter
        dict_common_interest.append(dict_common_buf)
    return dict_common_interest

In [220]:
dict_common_interest2 = get_common_interest('interest2', 0.25)
df_common_interest2 = pd.DataFrame(dict_common_interest2)
X_train = pd.merge(X_train, df_common_interest2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest2, on=['aid'], how='left')

In [221]:
dict_common_interest1 = get_common_interest('interest1', 0.25)
df_common_interest1 = pd.DataFrame(dict_common_interest1)
X_train = pd.merge(X_train, df_common_interest1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest1, on=['aid'], how='left')

In [222]:
dict_common_interest5 = get_common_interest('interest5', 0.25)
df_common_interest5 = pd.DataFrame(dict_common_interest5)
X_train = pd.merge(X_train, df_common_interest5, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest5, on=['aid'], how='left')

In [223]:
X_train['num_common_interest2'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest2', 'common_interest2']].values]
X_test['num_common_interest2'] = [len(set(i.split(' ')).intersection(set(j))) /len(j) for i, j in X_test[['interest2', 'common_interest2']].values]

In [224]:
X_train['num_common_interest1'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest1', 'common_interest1']].values]
X_test['num_common_interest1'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_test[['interest1', 'common_interest1']].values]

In [225]:
X_train['num_common_interest5'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest5', 'common_interest5']].values]
X_test['num_common_interest5'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_test[['interest5', 'common_interest5']].values]

In [226]:
dict_common_topic1 = get_common_interest('topic1', 0.1)
df_common_topic1 = pd.DataFrame(dict_common_topic1)
X_train = pd.merge(X_train, df_common_topic1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_topic1, on=['aid'], how='left')
X_train['num_common_topic1'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['topic1', 'common_topic1']].values ]
X_test['num_common_topic1'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['topic1', 'common_topic1']].values ]

In [227]:
X_train_encoded = ct_trains
X_test_encoded = ct_tests

In [228]:
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(data[feature].reshape(-1, 1))
    train_a=oc_encoder.transform(X_train[feature].values.reshape(-1, 1))
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a=oc_encoder.transform(X_test[feature].values.reshape(-1, 1))
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [229]:
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']
ct_encoder = CountVectorizer(min_df=0.0009)
for feature in vector_feature:
    ct_encoder.fit(data[feature])
    train_a = ct_encoder.transform(X_train[feature])
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a = ct_encoder.transform(X_test[feature])
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

In [231]:
X_test_encoded.shape

(659912, 5114)

In [272]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[['num_advertise_touser', 'ratio_clicked','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 'num_common_topic1',
                      'ratio_num_ad_toage' ,'ratio_num_ad_toconsume', 'ratio_num_ad_tohouse', 'ratio_num_campaign']].values)
train_encoded = scaler.transform(X_train[['num_advertise_touser', 'ratio_clicked', 'num_common_interest2', 'num_common_interest1', 'num_common_interest5', 
                      'num_common_topic1', 'ratio_num_ad_toage', 'ratio_num_ad_toconsume', 'ratio_num_ad_tohouse', 'ratio_num_campaign']].values)

test_encoded = scaler.transform(X_test[['num_advertise_touser', 'ratio_clicked','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 
                        'num_common_topic1', 'ratio_num_ad_toage', 'ratio_num_ad_toconsume', 'ratio_num_ad_tohouse', 'ratio_num_campaign']].values)

# scaler.fit(X_train[['num_advertise_touser']].values)
# train_encoded = scaler.transform(X_train[['num_advertise_touser']].values)
# test_encoded = scaler.transform(X_test[['num_advertise_touser']].values) 
# ,'num_common_interest2', 'num_common_interest1', 'num_common_interest5', 'num_common_topic1'  'ratio_num_ad_toconsume' 
X_train_encoded2 = sparse.hstack((X_train_encoded, train_encoded))
X_test_encoded2 = sparse.hstack((X_test_encoded, test_encoded))

# X_train_encoded2 = X_train_encoded
# X_test_encoded2 = X_test_encoded

In [147]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

model = SelectKBest(chi2, k=4000).fit(X_train_encoded, y_train)
X_train_new = model.transform(X_train_encoded)
X_test_new = model.transform(X_test_encoded)

训练用卡方检验选取的特征

In [148]:
X_train_encoded3 = sparse.hstack((X_train_new, train_encoded))
X_test_encoded3 = sparse.hstack((X_test_new, test_encoded))

In [273]:
# y_train = np.array(target).squeeze()
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=0.9,
        max_depth=-1, n_estimators=150, objective='binary',
        subsample=0.7, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.05, min_child_weight=60, random_state=2018, n_jobs=-1
    )
clf.fit(X_train_encoded2, y_train, eval_set=[(X_test_encoded2, y_test)], eval_metric='auc',early_stopping_rounds=50)

[1]	valid_0's auc: 0.63366
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.636505
[3]	valid_0's auc: 0.640027
[4]	valid_0's auc: 0.649399
[5]	valid_0's auc: 0.649131
[6]	valid_0's auc: 0.651876
[7]	valid_0's auc: 0.652166
[8]	valid_0's auc: 0.653423
[9]	valid_0's auc: 0.654501
[10]	valid_0's auc: 0.655838
[11]	valid_0's auc: 0.656319
[12]	valid_0's auc: 0.658282
[13]	valid_0's auc: 0.658428
[14]	valid_0's auc: 0.658616
[15]	valid_0's auc: 0.659008
[16]	valid_0's auc: 0.659754
[17]	valid_0's auc: 0.659946
[18]	valid_0's auc: 0.660957
[19]	valid_0's auc: 0.661452
[20]	valid_0's auc: 0.662046
[21]	valid_0's auc: 0.661842
[22]	valid_0's auc: 0.661727
[23]	valid_0's auc: 0.662151
[24]	valid_0's auc: 0.663517
[25]	valid_0's auc: 0.663133
[26]	valid_0's auc: 0.66321
[27]	valid_0's auc: 0.663319
[28]	valid_0's auc: 0.663621
[29]	valid_0's auc: 0.663832
[30]	valid_0's auc: 0.664211
[31]	valid_0's auc: 0.664484
[32]	valid_0's auc: 0.664718
[33]	valid_0's auc: 0

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=60, min_split_gain=0.0, n_estimators=150,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.9, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1)

第一步参数调整，固定学习速率为0.05, 正则化参数默认, 

In [275]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp,space_eval,rand,Trials,partial,STATUS_OK

def objective(argsDict):
    num_leaves = argsDict["num_leaves"] * 10 + 10
    n_estimators = argsDict['n_estimators'] * 10 + 50
#     learning_rate = argsDict["learning_rate"] * 0.02 + 0.05
    subsample = argsDict["subsample"] * 0.1 + 0.7
    colsample_bytree = argsDict["colsample_bytree"] * 0.1 + 0.7
    scale_pos_weight = argsDict["scale_pos_weight"] + 1
    min_child_weight = argsDict["min_child_weight"] * 10 + 20
    
    print("num_leaves:" + str(num_leaves))
    print("n_estimator:" + str(n_estimators))
    print("subsample:" + str(subsample))
    print("colsample_bytree:" + str(colsample_bytree))
    print("scale_pos_weight:" + str(scale_pos_weight))
    print("min_child_weight:" + str(min_child_weight))

    gbm = lgb.LGBMClassifier(boosting_type='gbdt', 
                             objective='binary',
                             learning_rate=0.1, 
                             num_leaves=num_leaves, 
                             reg_alpha=0.0, 
                             reg_lambda=1,
                             max_depth=-1,
                             n_estimators=n_estimators, 
                             subsample=subsample, 
                             colsample_bytree=colsample_bytree, 
                             subsample_freq=1,
                             min_child_weight=min_child_weight, 
                             scale_pos_weight=scale_pos_weight, 
                             random_state=2018, n_jobs=-1)
    gbm.fit(X_train_encoded2, y_train, eval_set=[(X_test_encoded2, y_test)], eval_metric='auc',early_stopping_rounds=50)
    metric = gbm.best_score_['valid_0']['auc']
#     metric = cross_val_score(gbm, X_test_encoded2, y_train, cv=5,scoring="roc_auc").mean()
    print(metric)
    return -metric


In [277]:
space = {"num_leaves":hp.randint("num_leaves",10),
         "n_estimators":hp.randint("n_estimators", 50), 
         "subsample":hp.randint("subsample",4), 
         "colsample_bytree":hp.randint("colsample_bytree",4),
         "scale_pos_weight": hp.randint("scale_pos_weight", 20),
         "min_child_weight": hp.randint("min_child_weight",8),
        }

algo = partial(tpe.suggest,n_startup_jobs=1)
best = fmin(objective,space,algo=algo,max_evals=500)

print(best)

num_leaves:50
n_estimator:190
subsample:0.7999999999999999
colsample_bytree:0.7999999999999999
scale_pos_weight:13
min_child_weight:20
[1]	valid_0's auc: 0.656776
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.667452
[3]	valid_0's auc: 0.671668
[4]	valid_0's auc: 0.674301


KeyboardInterrupt: 